In [1]:
import os

def loadFAQs(directory_path):
   faqs = {}

   for filename in os.listdir(directory_path):
      if filename.endswith(".txt"):  # assuming FAQs are in .txt files
         file_path = os.path.join(directory_path, filename)

         with open(file_path) as f:
            raw_faq = f.read()

         filename_without_ext = os.path.splitext(filename)[0]  # remove .txt extension
         faqs[filename_without_ext] = [text.strip() for text in raw_faq.split('=====')]

   return faqs

In [2]:
faqs = loadFAQs('.')
faqs

{'requierements': ['oracledb\ntqdm\nipywidgets\nsentence_transformers\noci'],
 'links': ['https://apexapps.oracle.com/pls/apex/r/dbpm/livelabs/run-workshop?p210_wid=3939&p210_wec=&session=115486261083319\nhttps://medium.com/@deepalimittal23/accessing-oci-object-storage-using-instance-principals-203ac93cb77'],
 'faq': ['Who are you and what can you do?\n\nI am DORA, the Digital ORacle Assistant, a digital assistant working for Oracle EMEA. I can answer questions about Oracle Cloud (OCI) and especially about the Free Trial and Always Free programs.',
  'What is Oracle Cloud Free Tier?\n\nOracle Cloud Free Tier allows you to sign up for an Oracle Cloud account which provides a number of Always Free services and a Free Trial with US$300 of free credit to use on all eligible Oracle Cloud Infrastructure services for up to 30 days. The Always Free services are available for an unlimited period of time. The Free Trial services may be used until your US$300 of free credits are consumed or the 3

In [3]:
docs = [{'text': filename + ' | ' + section, 'path': filename} for filename, sections in faqs.items() for section in sections]

# Sample the resulting data
docs[:2]

[{'text': 'requierements | oracledb\ntqdm\nipywidgets\nsentence_transformers\noci',
  'path': 'requierements'},
 {'text': 'links | https://apexapps.oracle.com/pls/apex/r/dbpm/livelabs/run-workshop?p210_wid=3939&p210_wec=&session=115486261083319\nhttps://medium.com/@deepalimittal23/accessing-oci-object-storage-using-instance-principals-203ac93cb77',
  'path': 'links'}]

In [4]:
import oracledb

from tqdm.autonotebook import tqdm, trange



/tmp/ipykernel_3943/3746924790.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
connection = oracledb.connect(
    user="admin",
    password="Oracle123...",
    dsn="(description= (retry_count=20)(retry_delay=3)(address=(protocol=tcps)(port=1521)(host=adb.us-chicago-1.oraclecloud.com))(connect_data=(service_name=ga720f82cae05fc_zna1jlsyim9n00j0_high.adb.oraclecloud.com))(security=(ssl_server_dn_match=yes)))")  # the connection string copied from the cloud console

print("Successfully connected to Oracle Database")

Successfully connected to Oracle Database


In [6]:
table_name = 'faqs'

with connection.cursor() as cursor:
    # Create the table
    create_table_sql = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            id NUMBER PRIMARY KEY,
            payload CLOB CHECK (payload IS JSON),
            vector VECTOR
        )"""
    try:
        cursor.execute(create_table_sql)
    except oracledb.DatabaseError as e:
        raise

    connection.autocommit = True

In [7]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer('all-MiniLM-L12-v2')

model.safetensors:   8%|7         | 10.5M/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/home/ubuntu/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
import array

# Define a list to store the data
data = [
   {"id": idx, "vector_source": row['text'], "payload": row} 
   for idx, row in enumerate(docs)
]

# Collect all texts for batch encoding
texts = [f"{row['vector_source']}" for row in data]

# Encode all texts in a batch
embeddings = encoder.encode(texts, batch_size=32, show_progress_bar=True)

# Assign the embeddings back to your data structure
for row, embedding in zip(data, embeddings):
   row['vector'] = array.array("f", embedding)

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

In [9]:
import json

with connection.cursor() as cursor:
    # Truncate the table
    cursor.execute(f"truncate table {table_name}")

    prepared_data = [(row['id'], json.dumps(row['payload']), row['vector']) for row in data]

    # Insert the data
    cursor.executemany(
        f"""INSERT INTO {table_name} (id, payload, vector)
        VALUES (:1, :2, :3)""",
        prepared_data
    )

    connection.commit()

In [10]:
with connection.cursor() as cursor:
    # Define the query to select all rows from a table
    query = f"SELECT * FROM {table_name}"

    # Execute the query
    cursor.execute(query)

    # Fetch all rows
    rows = cursor.fetchall()

    # Print the rows
    for row in rows[:5]:
        print(row)

(122, {'text': 'faq | What is the Oracle Cloud Infrastructure Compute API, and how can it be used?\n\nThe Oracle Cloud Infrastructure Compute API allows developers to programmatically manage and interact with compute instances, including creating, modifying, and terminating instances.', 'path': 'faq'}, array('f', [0.0818793997168541, -0.0008638272411189973, -0.004283372312784195, -0.04060010239481926, -0.027023285627365112, -0.0007229556795209646, -0.007969026453793049, 0.0354604534804821, 0.016790721565485, 0.03819073736667633, 0.04670068249106407, -0.05441199615597725, 0.016023797914385796, -0.10824279487133026, -0.06405135989189148, -0.09402532875537872, 0.008396985940635204, 0.021330978721380234, -0.058389995247125626, -0.0515948086977005, 0.02285577915608883, 0.052265919744968414, -0.05111362040042877, -0.028827566653490067, 0.013363363221287727, -0.06274005770683289, 0.006875917781144381, 0.03890277445316315, 0.011837686412036419, 0.002121781464666128, 0.12987135350704193, 0.0031

In [11]:
topK = 3

sql = f"""select payload, vector_distance(vector, :vector, COSINE) as score
from {table_name}
order by score
fetch approx first {topK} rows only"""

In [12]:
question = 'My resources no longer appear. How can I restore them?'

In [13]:
with connection.cursor() as cursor:
  embedding = list(encoder.encode(question))
  vector = array.array("f", embedding)

In [14]:
with connection.cursor() as cursor:
  embedding = list(encoder.encode(question))
  vector = array.array("f", embedding)

  results  = []

  for (info, score, ) in cursor.execute(sql, vector=vector):
      text_content = info.read()
      results.append((score, json.loads(text_content)))

In [15]:
import pprint
pprint.pp(results)

[(0.2884202845201732,
  {'text': 'faq | My resources no longer appear. How can I restore them?\n'
           '\n'
           'If you have a Free Tier account and your resources no longer '
           'appear, it’s likely that your Free Trial has expired and your paid '
           'resources have been reclaimed (terminated). You can verify if this '
           'is the case by doing the following: \n'
           'Log in to the Console\n'
           'Check for a banner at the top of the Console with the following '
           'text: “You are using a Free Tier account. To access all services '
           'and resources, upgrade to a paid account.”\n'
           '\n'
           'If you see this message, your resources have been reclaimed and '
           'can’t be restored.',
   'path': 'faq'}),
 (0.6864412401442181,
  {'text': 'faq | How do I recover my data once the 30-day Free Trial comes to '
           'an end?\n'
           '\n'
           'You are responsible for exporting and import

In [16]:
from transformers import LlamaTokenizerFast
import sys

tokenizer = LlamaTokenizerFast.from_pretrained("hf-internal-testing/llama-tokenizer")


tokenizer.model_max_length = sys.maxsize

def truncate_string(string, max_tokens):
    # Tokenize the text and count the tokens
    tokens = tokenizer.encode(string, add_special_tokens=True) 
    # Truncate the tokens to a maximum length
    truncated_tokens = tokens[:max_tokens]
    # transform the tokens back to text
    truncated_text = tokenizer.decode(truncated_tokens)
    return truncated_text

tokenizer_config.json:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [17]:
# transform docs into a string array using the "paylod" key
docs_as_one_string = "\n=========\n".join([doc["text"] for doc in docs])
docs_truncated = truncate_string(docs_as_one_string, 1000)

In [18]:
prompt = f"""\
    <s>[INST] <<SYS>>
    You are a helpful assistant named Oracle chatbot. 
    USE ONLY the sources below and ABSOLUTELY IGNORE any previous knowledge.
    Use Markdown if appropriate.
    Assume the customer is highly technical.
    <</SYS>> [/INST]

    [INST]
    Respond to PRECISELY to this question: "{question}.",  USING ONLY the following information and IGNORING ANY PREVIOUS KNOWLEDGE.
    Include code snippets and commands where necessary.
    NEVER mention the sources, always respond as if you have that knowledge yourself. Do NOT provide warnings or disclaimers.
    =====
    Sources: {docs_truncated}
    =====
    Answer (Three paragraphs, maximum 50 words each, 90% spartan):
    [/INST]
    """

In [19]:
import oci

In [20]:
compartment_id = "ocid1.tenancy.oc1..aaaaaaaaogia3wprum3yvc5qhcrdcxzrtwloiusgk5dbb5uibgcayapljwtq"

# Crear un proveedor de autenticación de instancias principales
signer = oci.auth.signers.InstancePrincipalsSecurityTokenSigner()

# Service endpoint
endpoint = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"

generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(config={}, signer=signer, service_endpoint=endpoint, retry_strategy=oci.retry.NoneRetryStrategy(), timeout=(10,240))
chat_detail = oci.generative_ai_inference.models.ChatDetails()

content = oci.generative_ai_inference.models.TextContent()
content.text = f"{prompt}"
message = oci.generative_ai_inference.models.Message()
message.role = "USER"
message.content = [content]
chat_request = oci.generative_ai_inference.models.GenericChatRequest()
chat_request.api_format = oci.generative_ai_inference.models.BaseChatRequest.API_FORMAT_GENERIC
chat_request.messages = [message]
chat_request.max_tokens = 1000
chat_request.temperature = 1
chat_request.frequency_penalty = 0
chat_request.presence_penalty = 0
chat_request.top_p = 0.75
chat_request.top_k = -1

chat_detail.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(model_id="ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceyarleil5jr7k2rykljkhapnvhrqvzx4cwuvtfedlfxet4q")
chat_detail.chat_request = chat_request

chat_detail.compartment_id = compartment_id
chat_response = generative_ai_inference_client.chat(chat_detail)
# Print result
print("**************************Chat Result**************************")
#print(vars(chat_response))

**************************Chat Result**************************


In [21]:
print(chat_response.data.chat_response.choices[0].message.content[0].text)

### Restoring Resources

If your resources no longer appear, you may need to re-authenticate or re-initialize your Oracle Cloud Infrastructure (OCI) configuration. Run the command `oci iam compartment list` to verify your compartment settings.

### Re-initializing OCI Configuration

To re-initialize your OCI configuration, run the command `oci setup config` and follow the prompts to re-enter your configuration settings. This will re-authenticate your session and restore access to your resources.

### Verifying Resources

After re-initializing your OCI configuration, run the command `oci resource search` to verify that your resources are restored. If issues persist, contact Oracle Cloud support for further assistance.
